## YFINANCE API 
    

In [1]:
!pip install yfinance


     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB 660.6 kB/s eta 0:00:05
     ---------------------------------------- 0.0/3.0 MB 660.6 kB/s eta 0:00:05
     - -------------------------------------- 0.1/3.0 MB 751.6 kB/s eta 0:00:04
     - -------------------------------------- 0.1/3.0 MB 774.0 kB/s eta 0:00:04
     -- ------------------------------------- 0.2/3.0 MB 888.4 kB/s eta 0:00:04
     ----- ---------------------------------- 0.4/3.0 MB 1.5 MB/s eta 0:00:02
     -------- ------------------------------- 0.7/3.0 MB 2.1 MB/s eta 0:00:02
     ------------ --------------------------- 1.0/3.0 MB 2.7 MB/s eta 0:00:01
     ---------------- ----------------------- 1.2/3.0 MB 3.1 MB/s eta 0:00:01
     -------------------- ------------------- 1.5/3.0 MB 3.4 MB/s eta 0:00:01
     ----------------------- ---------------- 1.8/3.0 MB 3.6 MB/s eta 0:00:01
     --------------------------- ------------ 2.1/3.0 MB 3.8 

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np


In [ ]:
#This script will pull data from yfinance's API, save as csv and send to a DB
def get_sp500_data(start_date, end_date):
    def update_data(df,fetched_dfs):
        for fetched_df in fetched_dfs:
            fetched_df = fetched_df.transpose()
            fetched_df['Ticker'] = df['Ticker']
            df = pd.concat([df, fetched_df], axis=0)
        return df
        
    # Use Wikipedia list for reference
    sp500_tickers = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]

    symbols = sp500_tickers['Symbol'].tolist()
    # Get the stock data for all the stocks in the S&P 500 index from 2010 to 2020
    # Create an empty DataFrame to store the data
    data = pd.DataFrame()

    # Loop through each ticker and get the data
    for ticker in symbols:
        try:
            # Get the stock data for the ticker
            stock_data = yf.Ticker(ticker).history(start=start_date, end=end_date)
            # Add a column for the ticker symbol
            stock_data['Ticker'] = ticker
            # Append the data to the DataFrame
            data = pd.concat([data, stock_data], axis=0)
            # Now we do income statement
            income_statement = yf.Ticker(ticker).get_income_statement()
            # Now we do balance sheet
            balance_sheet = yf.Ticker(ticker).get_balance_sheet()
            # Now we do cash flow
            cash_flow = yf.Ticker(ticker).get_cashflow()
            fetched_dfs = [income_statement, balance_sheet, cash_flow]
            # Update the data with the fetched data
            data = update_data(data, fetched_dfs)
        
        except Exception as e:
            print(f"Error retrieving data for {ticker}: {e}")

    data.reset_index(inplace=True)

    # add the details from sp500_tickers to data
    data = data.merge(sp500_tickers, left_on='Ticker', right_on='Symbol', how='left')

data = get_sp500_data()
# Save the data to a CSV file
data.to_csv("sp500_data.csv", index=False)
data.head()

$BRK.B: possibly delisted; no timezone found
C:\Users\georg\AppData\Local\Temp\ipykernel_40560\3435568841.py:22: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  data = pd.concat([data, stock_data], axis=0)
$BF.B: possibly delisted; no price data found  (1d 2010-01-01 -> 2020-12-31)
C:\Users\georg\AppData\Local\Temp\ipykernel_40560\3435568841.py:22: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  data = pd.concat([data, stock_data], axis=0)
$COIN: possibly delisted; no price data found  (1d 2010-01-01 -> 2020-12-31) (Yahoo error = "Data doesn't exist for star

KeyboardInterrupt: 

In [ ]:
def get_SP500_data_macro(start,end):
        """
        Fetches historical stock data for the overall S&P 500 from YFinance.
        """
        # Fetch the S&P 500 data
        sp500 = yf.Ticker("^GSPC")
        # Get the historical data for the S&P 500
        sp500_data = sp500.history(start=start, end=end)
        return sp500_data

In [ ]:
data_macro = get_SP500_data_macro("2010-01-01", "2020-12-31")
# Save the macro data to a CSV file
data_macro.to_csv("sp500_macro_data.csv", index=False)
data_macro.head()
